In [15]:
import math
import altair as alt
from altair import datum
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from vega_datasets import data
from datetime import datetime
import calendar

In [16]:
'ipywidgets ' + widgets.__version__ + ', altair ' + alt.__version__ + ', pandas ' + pd.__version__ + ', numpy ' + np.__version__

'ipywidgets 8.0.4, altair 5.1.2, pandas 2.0.3, numpy 1.24.3'

In [17]:
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [18]:
# loading the csv file into a dataframe
bos_weather_df = pd.read_csv("boston-weather-mid.csv")
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

yearCol = 'year'

# initializing the year range
yearRangeInData=[bos_weather_df[yearCol].min(),
                 bos_weather_df[yearCol].max()]

bos_weather_df

,year,month,maxTemp,minTemp,meanTemp,precipitation,CDDBase65
0,1872,Jan,NaN,NaN,NaN,2.0,NaN
1,1872,Feb,NaN,NaN,NaN,2.0,NaN
2,1872,Mar,48.0,-8.0,18.0,4.0,0.0
3,1872,Apr,85.0,28.0,38.0,1.0,1.0
4,1872,May,85.0,40.0,51.0,3.0,16.0
...,...,...,...,...,...,...,...
1816,2023,May,86.0,42.0,51.0,2.0,39.0
1817,2023,Jun,84.0,47.0,59.0,3.0,106.0
1818,2023,Jul,91.0,61.0,68.0,10.0,332.0
1819,2023,Aug,88.0,59.0,65.0,6.0,216.0


In [19]:
# source: https://stackoverflow.com/questions/48042915/sort-a-pandas-dataframe-series-by-month-name
# source: https://vega.github.io/vega/docs/schemes/
# source: https://stackoverflow.com/questions/69436980/change-thickness-of-one-line-on-altair-chart
# source: https://altair-viz.github.io/gallery/line_chart_with_points.html
# source: https://stackoverflow.com/questions/57695261/changing-the-size-of-altair-plot-renders-in-jupyter-notebook
# source: https://stackoverflow.com/questions/66158128/change-legend-number-range-in-altair-plot

# IS THERE A TOOL TIP???????????????

# setting up the slider widget
@interact(
    yearRange=widgets.IntRangeSlider(
        value=yearRangeInData,
        min=yearRangeInData[0],
        max=yearRangeInData[1],
        step=1,
        description='Years:'
    )
)
def lineChart(yearRange):
    """
    Creates a line chart displaying the mean temp changing over the months in the given range of years
    yearRange: a widget that sets the range of years the visualization will show data from
    
    returns an altair time-series chart
    """
    # Filtering the df based on the year range set in the slider
    year_range_df = bos_weather_df[(bos_weather_df['year'] >= yearRange[0]) & (bos_weather_df['year'] <= yearRange[1])]

    # Returning the chart
    return alt.Chart(year_range_df, title = alt.Title(
                "How the Boston temperatures for each month have changed over time"
                 
                # updating subtitle based on year range
                , subtitle='A comparison of '+ str(yearRange[0]) + '-' + str(yearRange[1])
                )).transform_filter(
                        (datum[yearCol] >= (yearRangeInData[0]) &
                        (datum[yearCol] <= (yearRange[1])))

                # Setting line and point marks
                ).mark_line(point = True

                # setting axes values
                ).encode(
                    x=alt.X('month').sort(months),

                    # y-axis staying the same
                    y= alt.Y('meanTemp', title = "mean monthly temp. in degrees Fahrenheit",
                        scale=alt.Scale(domain=[5, 73]),
                        axis=alt.Axis(tickCount=20)),
        
                    # setting up tooltip so additional info is shown when you hover over a point
                    tooltip=['year', 'month', 'minTemp', 'maxTemp', 'meanTemp', 'precipitation'],
        
                    # setting up legend
                    color=alt.Color('year', scale=alt.Scale(scheme='lightgreyred', domain=[1872, 2023])),
                    strokeWidth=alt.value(0.5)
               
                # setting chart dimensions
                ).properties(
                    width=550,
                    height=600
                    )
   


interactive(children=(IntRangeSlider(value=(1872, 2023), description='Years:', max=2023, min=1872), Output()),…

In [20]:
# sources: https://stackoverflow.com/questions/62281179/how-to-adjust-scale-ranges-in-altair
# source: https://stackoverflow.com/questions/59699412/altair-display-all-axis-ticks-but-only-some-tick-labels

# setting up the dropdown widget
@interact(month=widgets.Dropdown(
    options=list(set(bos_weather_df['month'].tolist())),
    value='Jan',
    description='Month:',
))

def scatterPlot(month):
    """
    Creates a scatter chart displaying the mean temp for a selected month changing across the years
    month: a widget that sets the month 
    
    returns an altair layered chart of the median line and scatter plot
    """
 
    # filtering to the month that's set by the widget
    month = str(month)
    vis2_df = bos_weather_df.loc[bos_weather_df['month'] == month]
    brush = alt.selection_interval(encodings=['x'])
  
    # initializing the scatter chart
    points = alt.Chart(vis2_df, title=alt.Title("How the Boston " + month + " temperature has changed over time"

                # updating subtitle based on year range
            , subtitle="A comparison of 1872-2023 with interactive selection mean across years"
            )).mark_point().encode(
        # setting axes values
        x = alt.X('year', scale=alt.Scale(domain=[1860, 2040]), axis=alt.Axis(tickCount=20)),
        y= alt.Y('meanTemp', title = "mean monthly temp. in degrees Fahrenheit").scale(zero=False),

        # setting up tooltip so additional info is shown when you hover over a point
        tooltip=['year', 'month', 'minTemp', 'maxTemp', 'meanTemp', 'precipitation'],

        # setting up legend
        color=alt.Color('year', scale=alt.Scale(scheme='lightgreyred')),
        size=alt.condition(brush, alt.value(50), alt.value(1)),

        # setting chart dimensions
        ).properties(
            width=550,
            height=300
        ).add_params(
            brush
        )
    
    # initializing the mean line
    meanLine = alt.Chart().mark_rule(color='black', opacity=.5).encode(
        y=alt.Y('median(meanTemp):Q', scale=alt.Scale(zero=False)),
        size=alt.SizeValue(3)
        ).transform_filter(
            brush
        )

    return alt.layer(points, meanLine, data=vis2_df).transform_filter(
        datum.month == month
    )

interactive(children=(Dropdown(description='Month:', index=11, options=('Nov', 'Aug', 'Jun', 'Mar', 'Apr', 'Se…